# 选择运行环境
py37full


# 编译脚本

会使用 
/home/zzr/projectshare/nlfff/pynlfff/cnlfff/Makefile.p 多进程
/home/zzr/projectshare/nlfff/pynlfff/cnlfff/Makefile.s 单进程(就头不一样)

编译出
/home/zzr/projectshare/nlfff/pynlfff/cnlfff/compiled.parallel/relax4 多进程
/home/zzr/projectshare/nlfff/pynlfff/cnlfff/compiled.nonparallel/relax4 单进程


In [1]:
!/home/zxz/nlfff/code/cnlfff/init_compile_ps.sh

rm -f relax4
rm -f rebin3d.o
rm -rf *.o
rm -f relax1.o 
rm -f rebin prepro checkquality checkenergy saveboundary fftpot4 rebin.o prepro.o checkquality.o checkenergy.o saveboundary.o fftpot4.o
nvcc -lib -g -O0 --fmad=false optim.cu -o optim.o
optim.cu: In function ‘void green_(double*, double*, double*)’:
optim.cu:68:37: warning: format ‘%s’ expects argument of type ‘char*’, but argument 3 has type ‘char (*)[25]’ [-Wformat=]
     fscanf(initfile, "%s %i", &leer, &nx);
                           ~~~~~     ^
optim.cu:69:37: warning: format ‘%s’ expects argument of type ‘char*’, but argument 3 has type ‘char (*)[25]’ [-Wformat=]
     fscanf(initfile, "%s %i", &leer, &ny);
                           ~~~~~     ^
optim.cu:70:37: warning: format ‘%s’ expects argument of type ‘char*’, but argument 3 has type ‘char (*)[25]’ [-Wformat=]
     fscanf(initfile, "%s %i", &leer, &nz);
                           ~~~~~     ^
optim.cu:71:40: warning: format ‘%s’ expects argument of type ‘char*’, but argu



# 准备边界数据



In [5]:
from prepare_base import PrepareWorker

p_raw_path = r"/home/zxz/nlfff/data/raw/hmi.sharp_cea_720s.2007.20120907_000000_TAI.Bp.fits"
t_raw_path = r"/home/zxz/nlfff/data/raw/hmi.sharp_cea_720s.2007.20120907_000000_TAI.Bt.fits"
r_raw_path = r"/home/zxz/nlfff/data/raw/hmi.sharp_cea_720s.2007.20120907_000000_TAI.Br.fits"
save_root=r"/home/zxz/nlfff/data/pro"
# create object
pre_worker = PrepareWorker()
# pre
pre_worker.prepare_from_fits_Bprt(p_raw_path,t_raw_path,r_raw_path,save_root)

mkdir /home/zxz/nlfff/data/pro


True

# 运行全部流程

In [4]:
# 脚本地址  数据存储的文件夹 运行等级
!/home//zxz/nlfff/code/cnlfff/compiled.nonparallel/multigrid.sh  /home/zxz/nlfff/data/pro 1
# 运行等级可设置为： 1  2  3  12  123

BASE INFORMATION:
File name:        linff run log
App file name:    /home//zxz/nlfff/code/cnlfff/compiled.nonparallel/multigrid.sh
Compute platform: 
Byte Order:                      Little Endian
Linux vSun 5.4.0-126-generic #142-Ubuntu SMP Fri Aug 26 12:12:57 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
-----------------------------------------------
RUN LOG:
RUN grid:
1
start run date:
Thu 27 Oct 2022 05:47:38 PM CST
workspace:
/home/zxz/nlfff/data/pro


 argv[0]: 	 /home/zxz/nlfff/code/cnlfff/compiled.nonparallel/relax4
 argv[1]: 	 23
 argv[2]: 	 0
 calcb= 23 	 maxit= 0

 Ls=1.000000 	 mu0= 0.000001
 nx= 45, ny=100 , nz= 55 


 Size of double: 8
 nxnynz * Size of double: 1980000
 nxnynz * Size of float: 990000
 Computing Potential field 

 nx= 45, ny=100 , nz= 55 

 Vectormagnetogram loaded
 Only Bz is used for potential field 

 percent finished = 0.000000
 percent finished = 2.222222
 percent finished = 4.444444
 percent finished = 6.666667
 percent finished = 8.888889
 percent finish

# 只运行非线性无力场

In [ ]:
# 把程序移动到数据文件夹底下  因为他读取的是绝对路径
# 主程序
!cp /home/zzr/projectshare/workspace/code/cnlfff/compiled.nonparallel/relax4 /home/zzr/projectshare/workspace/data/pro
# rebin程序
!cp /home/zzr/projectshare/workspace/code/cnlfff/compiled.nonparallel/rebin /home/zzr/projectshare/workspace/data/pro

In [ ]:
# 下面在控制台运行
!cd /home/zzr/projectshare/workspace/data/pro

# 第一层  先产生势场 再非线性无力场
# 读取是绝对路径
!relax4 23 0
!relax4 40 10000

# 后面先rebin非线性无力场为边界条件  再非线性无力场
!rebin Bout.bin B0.bin
!relax4 40 10000
